In [76]:
import pandas as pd
import os

In [77]:
type_dict = {"id": "Int64","submitdate": "str", "lastpage": "string", "startlanguage": "string", "seed": "string", "token": "string", "startdate": "str", "datestamp": "str", "refurl": "string", "group": "string", "tasks": "Int64", "UUID": "string", "age": "string", "gender_mf": "string", "gender_other": "string", "education": "string", "usefulness[SQ001]": "Int64", "usefulness[SQ002]": "Int64", "usefulness[SQ003]": "Int64", "usefulness[SQ004]": "Int64", "usefulness[SQ005]": "Int64", "usefulness[SQ006]": "Int64", "ease_of_use[SQ001]": "Int64", "ease_of_use[SQ002]": "Int64", "ease_of_use[SQ003]": "Int64", "ease_of_use[SQ004]": "Int64", "ease_of_use[SQ005]": "Int64", "ease_of_use[SQ006]": "Int64", "self_efficacy[SQ001]": "Int64", "self_efficacy[SQ002]": "Int64", "self_efficacy[SQ003]": "Int64", "self_efficacy[SQ004]": "Int64", "self_efficacy[SQ005]": "Int64", "self_efficacy[SQ006]": "Int64", "self_efficacy[SQ007]": "Int64", "self_efficacy[SQ008]": "Int64", "self_efficacy[SQ009]": "Int64", "self_efficacy[SQ010]": "Int64", "load[SQ001]": "Int64", "load[SQ002]": "Int64", "load[SQ003]": "Int64", "load[SQ004]": "Int64", "load[SQ005]": "Int64", "load[SQ006]": "Int64"}

raw_res = pd.read_csv("./data/survey_results.csv", index_col="id", dtype=type_dict, parse_dates=["submitdate", "startdate", "datestamp"], date_format="%Y-%m-%dT%H:%M:%S%z")

In [84]:
working_df = raw_res.drop(columns=["lastpage", "startlanguage", "gender_other", "seed", "token", "refurl"])

expl_df = working_df[raw_res["group"] == "E"]
base_df = working_df[raw_res["group"] == "B"]

working_df

,submitdate,startdate,datestamp,group,tasks,UUID,age,gender_mf,education,usefulness[SQ001],...,self_efficacy[SQ007],self_efficacy[SQ008],self_efficacy[SQ009],self_efficacy[SQ010],load[SQ001],load[SQ002],load[SQ003],load[SQ004],load[SQ005],load[SQ006]
id,,,,,,,,,,,,,,,,,,,,,
1,2025-07-25 18:50:55+02:00,2025-07-25 18:26:03+02:00,2025-07-25 18:50:55+02:00,E,1,f971d5ae-a9ff-4a23-a92b-7650ad697dfc,30 - 39,Männlich,Master,7,...,3,1,8,9,10,1,3,15,10,4
2,2025-07-29 18:26:34+02:00,2025-07-29 18:18:52+02:00,2025-07-29 18:26:34+02:00,B,1,7d8c9331-dd8c-4df6-8f6c-9e7a16d008d9,20 - 29,Männlich,Allgemeine Hochschulreife (Abitur),2,...,9,9,9,9,3,8,0,6,12,12
3,2025-07-31 12:27:55+02:00,2025-07-31 12:21:54+02:00,2025-07-31 12:27:55+02:00,B,2,fa69270e-20e0-4849-bae1-473de60495a4,20 - 29,Männlich,Bachelor,7,...,9,9,9,10,18,8,0,19,15,15
4,2025-08-01 13:26:23+02:00,2025-08-01 13:18:45+02:00,2025-08-01 13:26:23+02:00,E,1,8d7e51b5-77bd-4474-8bcb-127fa69cd98c,20 - 29,Männlich,Allgemeine Hochschulreife (Abitur),7,...,10,9,10,10,2,0,0,18,0,0
5,2025-08-05 17:20:45+02:00,2025-08-05 17:09:23+02:00,2025-08-05 17:20:45+02:00,B,1,f41a9ecb-7000-4ae1-8b8e-1b39ea598476,20 - 29,Männlich,Bachelor,6,...,10,10,10,10,13,2,6,18,15,15
7,2025-08-05 18:40:54+02:00,2025-08-05 18:25:54+02:00,2025-08-05 18:40:54+02:00,E,2,adbea003-c17e-463f-8ae0-c397f6f3eec1,60 - 69,Männlich,Diplom,4,...,2,9,10,2,4,0,0,16,0,0
8,2025-08-07 11:42:23+02:00,2025-08-07 11:34:24+02:00,2025-08-07 11:42:23+02:00,E,2,74463bbb-d158-461a-82f3-a030a108e157,30 - 39,Männlich,Allgemeine Hochschulreife (Abitur),5,...,10,7,10,10,18,2,0,19,17,1
9,2025-08-08 11:37:57+02:00,2025-08-08 11:34:45+02:00,2025-08-08 11:37:57+02:00,E,1,c4806f2c-dd6d-425c-a23d-62d29b4a1e5b,20 - 29,Männlich,Bachelor,7,...,10,9,10,10,13,0,0,13,13,1
10,2025-08-12 12:02:01+02:00,2025-08-12 11:52:21+02:00,2025-08-12 12:02:01+02:00,E,1,dbc1d144-1c2c-42f5-b63c-e84c446a9189,20 - 29,Männlich,Bachelor,5,...,5,6,5,6,10,0,14,13,8,9


In [82]:
# produce separate output csv

os.makedirs("./out", exist_ok=True)

expl_df.to_csv("./out/survey_results_expl.csv")
base_df.to_csv("./out/survey_results_base.csv")